In [259]:
import pandas as pd
from sklearn import metrics
import numpy as np
import matplotlib.pyplot as plt

In [2]:
all_data = pd.read_csv('../data/all_data.csv').drop('Unnamed: 0', axis=1)

In [5]:
from statsmodels.tsa.arima.model import ARIMA

In [6]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima.model import ARIMA

In [492]:
# all_data.moex_index

In [346]:
# plot_pacf(all_data.eur_rub.diff().dropna());

In [347]:
# plot_acf(all_data.eur_rub.diff().dropna());

In [417]:
# plot_acf(all_data.nickel.diff().dropna());

In [493]:
# pacf(all_data.nickel.diff().dropna())[22]

In [494]:
# nickel_model = 

In [495]:
# forecast = model.get_forecast(steps=3)

In [496]:
# all_data.eur_rub.tail()

In [514]:
model = sm.tsa.SARIMAX(data.eur_rub, order=(1,1,1), trend='c').fit(disp=False)

In [498]:
model.simulate(nsimulations=3,anchor=889)

In [518]:
forecast = model.get_forecast(steps=1)

In [519]:
forecast.

# Code

In [297]:
def calculate_posterior_cov(
    factor_list: list=['factor_1', 'factor_2'],
    prior_cov: pd.DataFrame=pd.DataFrame(
        {'factor_1': [1, 1.1], 'factor_2': [1.1, 2]},
        index=['factor_1', 'factor_2']
    ),
    est_var=[1.1, 3],
):
    """
    склеиваем оценки дисперсий факторов и их ковариации
    """
    assert list(prior_cov.columns)==list(prior_cov.index), 'unexpected corr matrix index'

    LEN = len(factor_list)

    # создаем матрицу и добавляем оценки дисперсий факторов 
    res_cov = np.zeros(shape=(LEN, LEN)) + np.diag(est_var)

    # убираем лишнее и отсортировываем матрицу
    prior_cov = prior_cov.loc[factor_list, factor_list].to_numpy()

    # оставляем только ковариации
    only_cov = prior_cov - np.diag(np.diagonal(prior_cov))

    return res_cov + only_cov

In [414]:
import scipy.stats as sps

def sample_multivariate_normal(
    factor_list: list=['factor_1', 'factor_2'],
    prior_cov: pd.DataFrame=pd.DataFrame(
        {'factor_1': [1, 1.1], 'factor_2': [1.1, 2]},
        index=['factor_1', 'factor_2']
    ),
    est_var=[1.1, 3],
    est_mean: list=[10, 15],
    size=100,
    return_array=True,
):
    """
    В функцию подаем имена риск-факторов, средние и дисперсии в одном порядке!
    """
    posterior_cov = calculate_posterior_cov(factor_list, prior_cov, est_var)
    
    sample = sps.multivariate_normal(
        mean=est_mean,
        cov=posterior_cov,
    ).rvs(size=size)

    return sample if return_array else pd.DataFrame(sample, factor_list)

In [662]:
import math

def calculate_simulation_probability(
    factor_list: list,
    series_list: List[np.array],
    prior_cov: pd.DataFrame,
    est_var: List[np.array],
    est_mean: List[np.array],
    logging=False,
    logreturn=True,
):  
    """
    Returns smth similar to Log Likelihood s. t. Multivariate distribution
    using models individual forecasts and historical covariances
    """
    
    assert len(factor_list) == len(series_list), 'check number of factors'
    assert len(set(len(i) for array in series_list)) == 1, 'check num of days in provided series'

    prior_cov = prior_cov.loc[factor_list, factor_list]
    
    series_matrix = np.array(series_list).T
    est_var_matrix = np.array(est_var).T
    est_mean_matrix = np.array(est_mean).T

    LENGTH = series_matrix.shape[0]
    prob_list = [-1] * LENGTH
    
    for step in range(LENGTH):

        posterior_cov = calculate_posterior_cov(
            factor_list,
            prior_cov,
            est_var_matrix[step, :]
        )

        dist = sps.multivariate_normal(
            mean=est_mean_matrix[step, :],
            cov=posterior_cov
        )

        prob_list[step] = dist.pdf(series_matrix[step, :])

    if logging:
        print(prob_list)
        
    return np.sum(np.log(prob_list)) if logreturn else np.prod(prob_list)

In [579]:
from itertools import product
import warnings
from tqdm import tqdm

def fit_best_sarima_model(
    series, 
    seasonal = 0,
    ps = range(1, 3),
    d = [0, 1],
    qs = range(0, 3),
    Ps = range(0, 2),
    D= [0, 1],
    Qs = range(0, 2),
):
    grid = [ps, d, qs]

    if seasonal:
        grid += [Ps, D, Qs]
    
    parameters_list = list(product(*grid))
    results = []
    best_aic = float("inf")
    warnings.filterwarnings('ignore')

    
    for param in parameters_list:
        #try except нужен, потому что на некоторых наборах параметров модель не обучается
        order_list = (*param[:3],)
        seasonal_list = (*param[3:7],) if seasonal else (0, 0, 0, 0)
        
        try:
            model_sm = sm.tsa.SARIMAX(
                series,
                order=order_list, 
                seasonal_order=seasonal_list,
            ).fit(disp=-1)
            
        #выводим параметры, на которых модель не обучается и переходим к следующему набору
        except ValueError:
            print('wrong parameters:', param)
            continue
        aic = model_sm.aic
        
        #сохраняем лучшую модель, aic, параметры
        if aic < best_aic:
            best_model = model_sm
            best_aic = aic
            best_param = param
            
        results.append([param, model_sm.aic])
    
    return best_model, best_param

# Lets check

In [445]:
data = all_data.set_index('date')

In [488]:
train = data[data.index < '2023-12-01']

In [447]:
factor_list = ['aluminum', 'brent', 'eur_rub']

In [504]:
cov_matrix = train.corr()

In [580]:
model_dict = {}
param_dict = {}
for f in tqdm(factor_list):

    model_dict[f], param_dict[f] = fit_best_sarima_model(train[f])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.69it/s]


In [581]:
param_dict

{'aluminum': (2, 1, 2), 'brent': (2, 1, 2), 'eur_rub': (1, 1, 1)}

## 1 step

In [544]:
one_step_means = []
one_step_var = []

for name in factor_list:

    pred = model_dict[name].get_forecast(steps=1)

    one_step_means.append(pred.predicted_mean.values[0])
    one_step_var.append(pred.var_pred_mean.values[0])

In [574]:
sample = sample_multivariate_normal(
    factor_list=factor_list,
    prior_cov=cov_matrix,
    est_mean=one_step_means,
    est_var=one_step_var,size=5000        
)

sample.mean(axis=0)

array([2198.34883185,   80.65012768,   97.76141347])

In [575]:
data[factor_list].loc['2023-12-01']

aluminum    2209.0000
brent         78.8800
eur_rub       96.8827
Name: 2023-12-01, dtype: float64

# N steps 

Можно конечно мб какую-то векторную авторегрессию сюда накрутить, но имхо оверкилл

In [576]:
STEPS = 10

means_series = []
var_series = []

for name in factor_list:

    pred = model_dict[name].get_forecast(steps=STEPS)

    means_series.append(pred.predicted_mean.values)
    var_series.append(pred.var_pred_mean.values)

Потестим фукцию для "вероятности" траектории

In [594]:
# нужен численный индекс последней записи на трейне +1
ANCHOR = train['brent'].shape[0]

In [599]:
simulation_1 = []

for name in factor_list:

    simulation_1.append(
        model_dict[name].simulate(nsimulations=STEPS,anchor=ANCHOR).values
    )

In [612]:
# simulation_1

In [667]:
simulation_1

p = calculate_simulation_probability(
    factor_list,
    simulation_1,
    cov_matrix,
    var_series,
    means_series,
    logging=False,
    logreturn=True
)

In [654]:
0.01**10

1.0000000000000002e-20

In [596]:
# model_dict['brent'].simulate(nsimulations=3,anchor=ANCHOR)

In [542]:
sample = sps.multivariate_normal(
        mean=[5, 10],
        cov=[[4, 0], [0, 1]],
    )

In [543]:
sample.pdf([6, 11])

0.04259475109761325

In [539]:
sample.pdf([4, 9])

0.04259475109761325

In [506]:
# sample_multivariate_normal()

In [279]:
prior_cov=pd.DataFrame(
        {'factor_1': [1, 0], 'factor_2': [0, 2]},
        index=['factor_1', 'factor_2']
    )

df = prior_cov.loc[['factor_1', 'factor_2'], ['factor_1', 'factor_2']].to_numpy()



In [280]:
df - np.diag(np.diagonal(df))

array([[0, 0],
       [0, 0]])

In [256]:
sample_multivariate_normal()

In [263]:
all_data.set_index('date').cov()['aluminum']['brent']

6956.293793586401

In [267]:
all_data.set_index('date').cov().to_numpy()

array([[ 1.80364843e+05,  6.95629379e+03,  6.97002463e+02,
        -2.42813658e+02,  5.26221953e+04,  1.57413858e+06,
         2.43246554e+04, -9.71246341e+01, -2.80720013e+03,
        -2.78639630e+03, -1.47497902e+03, -2.27753016e+03,
        -3.26893345e+03,  1.42847403e+04,  1.83095649e+03,
         7.75151812e+04,  2.04047168e+03,  1.25961012e+05,
         7.92460652e+02,  1.42281119e+04,  5.25586812e+03,
         1.55313740e+03, -1.24073212e-02],
       [ 6.95629379e+03,  4.61717870e+02,  4.78175599e+01,
        -7.18167832e+01, -2.52909487e+03,  8.43027423e+04,
        -7.05330607e+02, -1.71625556e+01, -2.08788784e+02,
        -2.12126008e+02, -8.28975402e+01, -1.58816004e+02,
        -2.55178426e+02,  2.56383655e+02, -2.61992732e+02,
        -9.89846110e+02, -1.26102539e+01,  5.36921619e+03,
        -1.64290299e+02,  2.10106053e+02,  1.73356213e+02,
        -4.29356404e+02, -1.18383709e-01],
       [ 6.97002463e+02,  4.78175599e+01,  1.15457541e+01,
         6.03869243e+00, -3.9

In [266]:
# all_data.set_index('date').corr()['aluminum']['aluminum']